In [86]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import os
import re
import requests
import time
pd.set_option('max_rows',1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
response = requests.get("https://www.rottentomatoes.com/browse/opening")
content = response.content
soup = BeautifulSoup(content, "html.parser")
title = soup.find_all("div",attrs={"class": "media-list__movie-info"})
find_all = pd.DataFrame({'title':title})
find_all['url'] = fi
re.search(r"<a href=\"/m.*?\">", str(find_all.title[0])).group(0)

In [19]:
os.environ['seleniumDriver'] = '/home/haddy/selenium_firefox/drivers/geckodriver'

In [20]:
os.getenv('seleniumDriver')

'/home/haddy/selenium_firefox/drivers/geckodriver'

In [2]:
def clean_data(x):
    
    data = re.sub(r"<.*?>|[^a-zA-Z0-9 ]+",'',str(x)).strip()
    return data

In [49]:
# Get movies opening in Cinemas
def opening():
    
    
    response = requests.get("https://www.rottentomatoes.com/browse/opening")
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    title = soup.find_all("div",attrs={"class": "media-list__title"})
    #Some Movies are without tomato meter
    #score = soup.find_all("span", attrs={"class": "tMeterScore"})
    title_df = pd.DataFrame({"title":title})
    title_df.title = title_df.title.apply(lambda x: re.sub(r"<.*?>|\(.*?\)|[^a-zA-Z0-9 ]+",'',str(x)).strip())
    
    return title_df

In [33]:
#Get Upcoming Movies
def upcoming():
    ##### Web scrapper for infinite scrolling page #####
    driver = webdriver.Firefox(executable_path=os.getenv('seleniumDriver'))
    driver.get("https://www.rottentomatoes.com/browse/upcoming")
    time.sleep(5)  # Allow 5 seconds for the web page to open
    scroll_pause_time = 5 
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    upcoming_soup = BeautifulSoup(driver.page_source, "html.parser")
    upcoming_df = pd.DataFrame({'title': upcoming_soup.find_all("div", attrs={'class':'media-list__title'})})
    upcoming_df.title = upcoming_df.title.apply(lambda x: re.sub(r"<.*?>|[^a-zA-Z0-9 ]+",'',str(x)).strip())
    return upcoming_df

In [6]:
def grab_reviews(title):
    
    try:
        review_response = requests.get(f"https://www.rottentomatoes.com/m/{title}/reviews")
        review_content = review_response.content
        soup = BeautifulSoup(review_content, "html.parser")
        reviews = soup.find_all("div", attrs={"class":"the_review"})
        review_df = pd.DataFrame ({"reviews":reviews})
        review_df.reviews = review_df.reviews.apply(lambda x: re.sub(r"<.*?>|[^a-zA-Z0-9 ]+",'',str(x)).strip())
        return review_df.reviews.tolist()
    ###Broken URL
    except:
        print("Broken Url")
        return None

In [38]:
def MovieReview(title_df, upcoming=False):
    titles = title_df.title.apply(lambda x: x.replace(' ','_').lower())
    review_df = pd.DataFrame(columns=['title', 'reviews'])
    for title in titles:
        if upcoming:
            reviews = grab_reviews(title+'_2021') #grab only 2021 movies
        else:
            reviews = grab_reviews(title)
        for i in range(len(reviews)):
            if i%10 == 0:
                print(f"Sample Review:\n{title}: {reviews[i]}\n\n")
            review_df = review_df.append({'title':title.replace('_',' '), 'reviews':reviews[i]}, ignore_index=True)
    return review_df

In [50]:
opening_movies = opening() ; opening_movies

,title
0,The Forever Purge
1,The Boss Baby Family Business
2,Dead Pigs
3,Zola
4,Lydia Lunch The War Is Never Over
5,Summer of Soul
6,Till Death
7,First Date
8,The God Committee
9,The Phantom


In [34]:
upcoming_movies = upcoming() ; upcoming_movies

,title
0,The Nest
1,The Price of Freedom
2,Black Widow
3,The Woman Who Ran
4,Dachra
...,...
165,The Marvels
166,Captain Marvel 2
167,Avatar 2
168,Aquaman and the Lost Kingdom


In [37]:
openingM_review = MovieReview(opening_movies)

Here--> 0                                     the_forever_purge
1                         the_boss_baby_family_business
2                                             dead_pigs
3                                                  zola
4                     lydia_lunch_the_war_is_never_over
5     summer_of_soul_or_when_the_revolution_could_no...
6                                            till_death
7                                            first_date
8                                     the_god_committee
9                                      long_story_short
10                                          the_phantom
11                                            whitetail
12                                              cousins
13                          scenes_from_an_empty_church
14     secret_agent_dingledorf_and_his_trusty_dog_splat
15                                         a_dim_valley
16                                        kid_candidate
Name: title, dtype: object
Sample Review

In [297]:
review_df.head()

,title,reviews
0,The Forever Purge,Its not smart but its a little smarter than th...
1,The Forever Purge,The Purge movies still want you to believe the...
2,The Forever Purge,The Forever Purge is adept at providing the vi...
3,The Forever Purge,Of all the highconcept horror movies that beca...
4,The Forever Purge,Even in light of such shortcomings The Forever...


In [299]:
upco.head()

,title,reviews
0,the nest,Very effective tension with a long slow creep ...
1,the nest,James Suttles insidious home invasion horror s...
2,the price of freedom,What makes The Price of Freedom a film that sh...
3,the price of freedom,An absorbing disturbing and scrupulously wellr...
4,black widow,A not your average Marvel comic book action movie


In [301]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [302]:
sid = SentimentIntensityAnalyzer()

In [350]:
def get_sentiments(df):
    
    movie_sentiment = pd.DataFrame(columns=['title', 'sentiment', 'reviews'])
    for title in (df.title.unique()):
        #print(title)
        review =", ".join(df[df.title==title]['reviews'].tolist())
        movie_sentiment = movie_sentiment.append({'title':title, 'reviews':review, 'sentiment':None}, ignore_index=True)
        #print(movie_sentiment.head())
        
        
    movie_sentiment['sentiment'] = movie_sentiment.reviews.apply(lambda x: sid.polarity_scores(x)['compound'])
    df['compound_score'] = df.reviews.apply(lambda x: sid.polarity_scores(x)['compound'])
    
        
    return df, movie_sentiment.drop(['reviews'], 1)

In [351]:
up_score, up_sent = get_sentiments(upco)

In [352]:
up_sent

,title,sentiment
0,the nest,-0.0220
1,the price of freedom,-0.1027
2,black widow,0.9969
